# Parse me if you can #

Sometimes when crawling we have to parse websites that turn out to be SaaS - i.e., there is a special JS application which renders documents and which is downloaded first. Therefore, data that is to be rendered initially comes in a proprietary format. One of the examples is Google Drive. Last time we downladed and parsed some files from GDrive, however, we didn't parse GDrive-specific file formats, such as google sheets or google slides.

Today we will learn to obtain and parse such data using Selenium - a special framework for testing web-applications.

## 1. Getting started

Let's try to load and parse the page the way we did before:

In [1]:
!pip install bs4

In [2]:
import requests
from bs4 import BeautifulSoup
resp = requests.get("https://mimo-international.com/MIMO/instrument-families.aspx")
soup = BeautifulSoup(resp.text, 'lxml')
print(soup.body.text[:1000])



              //<![CDATA[
              
        master = { };
        master.lookupSiteInUrlHostName = false;
        master.allScopesInstanceUrl = 'https://mimo-international.com/MIMO/~/';
        master.useForFront = true;
        master.applicationName = '';
        master.applicationAlias = '';
        master.scopeId = null;
        master.scopeName = '';
        master.scopeDescription = '';
        master.scopeAlias = '';
        master.pageFullName = 'Archimed.Web.UI.PortalPage';
        master.uiStateData = '';
        master.currentCulture = 'en-US';
        master.changeCultureAllowed = true;
        master.isPopup = false;
        master.instanceName = 'MIMO';
        master.userDisplayName = 'Utilisateur anonyme';
        master.historyLabel = 'History';
        master.instanceUrl = 'https://mimo-international.com/MIMO/';
        master.baseUrl = 'https://mimo-international.com/MIMO/';
        master.homeUrl = 'https://mimo-international.com/MIMO/d


As we see, the output is not what we expect. So, what can we do when a page is not being loaded right away, but is rather rendered by a script? Browser engines can help us get data. Let's try to load the same web-page, but do it in a different way: let's give some time to a browser to load the scripts and run them; and then will work with DOM (Document Object Model), but will get it from browser engine itself, not from BeautifulSoup.

Where do we get browser engine from? Simply installing a browser will do the thing. How do we send commands to it from code and retrieve DOM? Service applications called drivers will interpret out commands and translate them into browser actions.


For each browser engine suport you will need to:
1. install browser itself;
2. download 'driver' - binary executable, which passed commands from selenium to browser. E.g. [Gecko == Firefox](https://github.com/mozilla/geckodriver/releases), [ChromeDriver](http://chromedriver.storage.googleapis.com/index.html);
3. unpack driver into a folder under PATH environment variable. Or specify exact binary location.

### Download driver

And place it in any folder or under PATH env. variable.

### Install selenium

In [3]:
!pip install -U selenium

Requirement already up-to-date: selenium in /home/imad/anaconda3/lib/python3.7/site-packages (3.141.0)


In [6]:
from selenium import webdriver

### Launch browser

This will open browser window

In [11]:
os.getcwd()+'/chromedriver'


'/home/imad/PycharmProjects/SCAI-COORD-IMAD/chromedriver'

In [36]:
browser = webdriver.Firefox(executable_path="/home/imad/Downloads/geckodriver-v0.28.0-linux64/geckodriver")
# import os
# os.environ["PATH"] += os.getcwd()
# or explicitly
# browser = webdriver.Chrome(
#     executable_path='/home/imad/Downloads/chromedriver_linux64 (1)/chromedriver', 
# )

### Download the page

In [37]:
# navigate to page
browser.get('https://mimo-international.com/MIMO/instrument-families.aspx')
browser.implicitly_wait(5)  # wait 5 seconds

# select all text parts from document
elements = browser.find_element_by_id("InstrumentTypeLevel2_exact")
elements = elements.find_elements_by_tag_name("li")
# print(elements.text)
# note that if number differs from launch to launch this means better extend wait time
print("Elements found:", len(elements))

# oh no! It glues all the words!
# print("What if just a silly approach:", elements[0].text)
for i in range(len(elements)):
    print(elements[i].text)
# GDrive stores all text blocks word-by-word
# subnodes = elements[0].find_elements_by_css_selector("text")
# text = " ".join(n.text for n in subnodes)
# print("What if a smart approach:", text)

Elements found: 9
Wind instruments (21936)
Percussion instruments (12676)
Stringed instruments (10981)
Elements of musical instruments (3447)
Keyboard instruments (2300)
Mechanical instruments (493)
Other instruments (114)
Mirlitons (94)
Electronic instruments (55)


In [41]:
from selenium.webdriver.common.action_chains import ActionChains
browser = webdriver.Firefox(executable_path="/home/imad/Downloads/geckodriver-v0.28.0-linux64/geckodriver")
from time import sleep
import urllib
import os

for i in range(2,3):
    browser.get('https://mimo-international.com/MIMO/instrument-families.aspx')
    browser.implicitly_wait(5)
    elements = browser.find_element_by_id("InstrumentTypeLevel2_exact")
    elements = elements.find_elements_by_tag_name("li")
    actions = ActionChains(browser)
    el = elements[i]
    title = el.text
    if not os.path.exists(title):
        os.mkdir(title)
    
    actions.move_to_element(el).click().perform();
    browser.implicitly_wait(5)
    while True:
        pictures = browser.find_element_by_id("resultats")
        pictures = pictures.find_elements_by_tag_name("img")
#         print(pictures)
        for picture in pictures:
            try:
                src = picture.get_attribute('src')
                src = src[:src.rindex("&w=")]
                name = src[src.rindex('/'):]
                print(str(title)+name)
                urllib.request.urlretrieve(src, title+name)
            except:
                pass
        next_ = browser.find_element_by_css_selector("button[class='btn-link-like suivant']")
        actions = ActionChains(browser)
        t= next_.find_element_by_xpath("..")
        # get_attribute() method to obtain class of parent
        print("Parent class attribute: " + t.get_attribute("class"))
        actions.move_to_element(next_).click().perform();
        browser.implicitly_wait(10)
        sleep(5)
#     print(browser.find_element_by_id("InstrumentTypeLevel3_exact").text)
# browser.quit()

Stringed instruments (10981)/CMIM000027421.jpg
Stringed instruments (10981)/1423_1.jpg
Stringed instruments (10981)/5483.1.jpg
Stringed instruments (10981)/0505.jpg
Stringed instruments (10981)/CMIM000027823.jpg
Stringed instruments (10981)/CMIM000029523.jpg
Stringed instruments (10981)/3963.jpg
Stringed instruments (10981)/CMIM000022548.jpg
Stringed instruments (10981)/1991.061_1.jpg
Stringed instruments (10981)/ad6b0ea6ce0917d73e3e896f3fe36a70.jpg
Stringed instruments (10981)/6dc6eb4bd2f957a281ce8e417d753082.jpg
Stringed instruments (10981)/22f99be56944de7038d5be5a657ed481.jpg
Stringed instruments (10981)/4035.jpg
Stringed instruments (10981)/0946.jpg
Stringed instruments (10981)/0709a.jpg
Parent class attribute: 
Stringed instruments (10981)/0767.jpg
Stringed instruments (10981)/0641.jpg
Stringed instruments (10981)/0518.jpg
Stringed instruments (10981)/3115.jpg
Stringed instruments (10981)/0642.jpg
Stringed instruments (10981)/0709b.jpg
Stringed instruments (10981)/0775.jpg
Stringe

Stringed instruments (10981)/CMIM000012543.jpg
Stringed instruments (10981)/1389_1.jpg
Stringed instruments (10981)/CMIM000012799.jpg
Stringed instruments (10981)/2536.jpg
Stringed instruments (10981)/4611.jpg
Stringed instruments (10981)/4612.jpg
Stringed instruments (10981)/M3481_01.jpg
Stringed instruments (10981)/M2817_01.jpg
Stringed instruments (10981)/3441.jpg
Stringed instruments (10981)/3442.jpg
Stringed instruments (10981)/4500.jpg
Stringed instruments (10981)/3289.jpg
Parent class attribute: 
Stringed instruments (10981)/M2213_01.jpg
Stringed instruments (10981)/CMIM000010964.jpg
Stringed instruments (10981)/1369_1.jpg
Stringed instruments (10981)/2835_1.jpg
Stringed instruments (10981)/2494_1.jpg
Stringed instruments (10981)/1432_1.jpg
Stringed instruments (10981)/CMIM000033564.jpg
Stringed instruments (10981)/e0da0bc3330acfbfb1aa646e4394d3e4.jpg
Stringed instruments (10981)/CMIM000023192.jpg
Stringed instruments (10981)/0600.jpg
Stringed instruments (10981)/CMIM000027784.j

Stringed instruments (10981)/3949_1.jpg
Stringed instruments (10981)/2820.jpg
Stringed instruments (10981)/CMIM000011329.jpg
Stringed instruments (10981)/CMIM000014339.jpg
Stringed instruments (10981)/CMIM000014341.jpg
Stringed instruments (10981)/MFIM000026608.jpg
Stringed instruments (10981)/CMIM000024706.jpg
Stringed instruments (10981)/3657.jpg
Parent class attribute: 
Stringed instruments (10981)/3706.jpg
Stringed instruments (10981)/CMIM000013138.jpg
Stringed instruments (10981)/3853_1.jpg
Stringed instruments (10981)/MFIM000026563.jpg
Stringed instruments (10981)/MFIM000023068.jpg
Stringed instruments (10981)/MFIM000008305.jpg
Stringed instruments (10981)/CMIM000011820.jpg
Stringed instruments (10981)/CMIM000016121.jpg
Stringed instruments (10981)/MFIM000006704.jpg
Stringed instruments (10981)/CMIM000018408.jpg
Stringed instruments (10981)/0698.jpg
Stringed instruments (10981)/CMIM000031369.jpg
Stringed instruments (10981)/MFIM000027069.jpg
Stringed instruments (10981)/CMIM00001

Stringed instruments (10981)/M1974_01.jpg
Stringed instruments (10981)/4508.jpg
Stringed instruments (10981)/4289.jpg
Stringed instruments (10981)/4567.jpg
Stringed instruments (10981)/a4fb014b5c6e63e34449ab28d7c12add.jpg
Stringed instruments (10981)/0032762c.jpg
Stringed instruments (10981)/CMIM000023087.jpg
Stringed instruments (10981)/3626.jpg
Stringed instruments (10981)/4056.jpg
Stringed instruments (10981)/0503.jpg
Parent class attribute: 
Stringed instruments (10981)/1678.jpg
Stringed instruments (10981)/CMIM000012684.jpg
Stringed instruments (10981)/CMIM000012049.jpg
Stringed instruments (10981)/MFIM000026635.jpg
Stringed instruments (10981)/3995.jpg
Stringed instruments (10981)/CMIM000015208.jpg
Stringed instruments (10981)/MFIM000021558.jpg
Stringed instruments (10981)/5879e76231f744dfbd53663c230fd2ae.jpg
Stringed instruments (10981)/81869e6894d3adc34eba6f98f982c85e.jpg
Stringed instruments (10981)/CMIM000016860.jpg
Stringed instruments (10981)/1993.006_1.jpg
Stringed instrum

Stringed instruments (10981)/CMIM000013190.jpg
Stringed instruments (10981)/CMIM000030435.jpg
Stringed instruments (10981)/CMIM000014253.jpg
Stringed instruments (10981)/CMIM000013776.jpg
Stringed instruments (10981)/CMIM000013742.jpg
Stringed instruments (10981)/1394_1.jpg
Stringed instruments (10981)/JT0415-01.jpg
Stringed instruments (10981)/0480_1.jpg
Stringed instruments (10981)/RCM0149.jpg
Stringed instruments (10981)/70bbc9f6a6ed63a43df5568db05cdab0.jpg
Stringed instruments (10981)/CMIM000013865.jpg
Stringed instruments (10981)/CMIM000013422.jpg
Parent class attribute: 
Stringed instruments (10981)/CMIM000013428.jpg
Stringed instruments (10981)/CMIM000010149.jpg
Stringed instruments (10981)/4001.jpg
Stringed instruments (10981)/44308639366aa80bcbb7347b0b8cae6d.jpg
Stringed instruments (10981)/CMIM000011842.jpg
Stringed instruments (10981)/4053_1.jpg
Stringed instruments (10981)/CMIM000013590.jpg
Stringed instruments (10981)/1988_026_f.jpg
Stringed instruments (10981)/b55b3a6850f

...

Stringed instruments (10981)/258499071831.jpg
Stringed instruments (10981)/3510686.jpg
Stringed instruments (10981)/3466439.jpg
Stringed instruments (10981)/3514315.jpg
Stringed instruments (10981)/3466937.jpg
Parent class attribute: 
Stringed instruments (10981)/3468493.jpg
Stringed instruments (10981)/210265071831.jpg
Stringed instruments (10981)/2948837.jpg
Stringed instruments (10981)/0692.jpg
Stringed instruments (10981)/3748.jpg
Stringed instruments (10981)/4576.jpg
Stringed instruments (10981)/3737.jpg
Stringed instruments (10981)/3444.jpg
Stringed instruments (10981)/4114.jpg
Stringed instruments (10981)/4239.jpg
Stringed instruments (10981)/0460.jpg
Stringed instruments (10981)/4836.jpg
Stringed instruments (10981)/0388.jpg
Stringed instruments (10981)/0652.jpg
Stringed instruments (10981)/4560.jpg
Parent class attribute: 
Stringed instruments (10981)/3674.jpg
Stringed instruments (10981)/0954.jpg
Stringed instruments (10981)/3508.jpg
Stringed instruments (10981)/4924.jpg
Stri

Stringed instruments (10981)/0032265c.jpg
Stringed instruments (10981)/0032949c-0002.jpg
Stringed instruments (10981)/0032911c.jpg
Stringed instruments (10981)/0033467c.jpg
Stringed instruments (10981)/0033057c.jpg
Stringed instruments (10981)/0032308d.jpg
Stringed instruments (10981)/0032272c.jpg
Stringed instruments (10981)/0033469c.jpg
Stringed instruments (10981)/0032298c.jpg
Stringed instruments (10981)/0032276c.jpg
Parent class attribute: 
Stringed instruments (10981)/0033059c.jpg
Stringed instruments (10981)/0032278c.jpg
Stringed instruments (10981)/4363087.jpg
Stringed instruments (10981)/3513294.jpg
Stringed instruments (10981)/3510360.jpg
Stringed instruments (10981)/3510682.jpg
Stringed instruments (10981)/3603039.jpg
Stringed instruments (10981)/3513264.jpg
Stringed instruments (10981)/3993323.jpg
Stringed instruments (10981)/3619751.jpg
Stringed instruments (10981)/209678071831.jpg
Stringed instruments (10981)/3514509.jpg
Stringed instruments (10981)/3466479.jpg
Stringed i

Stringed instruments (10981)/4393.jpg
Stringed instruments (10981)/3234.jpg
Stringed instruments (10981)/0614.jpg
Stringed instruments (10981)/0668.jpg
Stringed instruments (10981)/3805.jpg
Stringed instruments (10981)/0667.jpg
Parent class attribute: 
Stringed instruments (10981)/4136.jpg
Stringed instruments (10981)/0564.jpg
Stringed instruments (10981)/0424.jpg
Stringed instruments (10981)/0710.jpg
Stringed instruments (10981)/0850.jpg
Stringed instruments (10981)/0953.jpg
Stringed instruments (10981)/0894.jpg
Stringed instruments (10981)/4584.jpg
Stringed instruments (10981)/3994.jpg
Stringed instruments (10981)/4391.jpg
Stringed instruments (10981)/3916.jpg
Stringed instruments (10981)/0837.jpg
Stringed instruments (10981)/0762.jpg
Stringed instruments (10981)/3804.jpg
Stringed instruments (10981)/4216.jpg
Parent class attribute: 
Stringed instruments (10981)/0648.jpg
Stringed instruments (10981)/4609.jpg
Stringed instruments (10981)/0381.jpg
Stringed instruments (10981)/0501.jpg


Stringed instruments (10981)/4225.jpg
Stringed instruments (10981)/4229.jpg
Stringed instruments (10981)/4237.jpg
Stringed instruments (10981)/3648.jpg
Stringed instruments (10981)/4290.jpg
Stringed instruments (10981)/4883.jpg
Stringed instruments (10981)/3446.jpg
Stringed instruments (10981)/0704.jpg
Stringed instruments (10981)/4317.jpg
Parent class attribute: 
Stringed instruments (10981)/4503.jpg
Stringed instruments (10981)/4527.jpg
Stringed instruments (10981)/4526.jpg
Stringed instruments (10981)/4528.jpg
Stringed instruments (10981)/4537.jpg
Stringed instruments (10981)/4548.jpg
Stringed instruments (10981)/4726.jpg
Stringed instruments (10981)/4729.jpg
Stringed instruments (10981)/4780.jpg
Stringed instruments (10981)/4781.jpg
Stringed instruments (10981)/4324.jpg
Stringed instruments (10981)/5623.jpg
Stringed instruments (10981)/2211.jpg
Stringed instruments (10981)/2380.jpg
Stringed instruments (10981)/4385.jpg
Parent class attribute: 
Stringed instruments (10981)/2279.jpg


NoSuchElementException: Message: Unable to locate element: button[class='btn-link-like suivant']


### Problems
- Too slow, wait for browser to open, browser to render

### Headless mode
Browsers (at least [FF](https://developer.mozilla.org/en-US/docs/Mozilla/Firefox/Headless_mode), [Chrome](https://intoli.com/blog/running-selenium-with-headless-chrome/), IE) have headless mode - no window rendering and so on. Means it should work much faster!

In [ ]:
# options = webdriver.FirefoxOptions()

# options.add_argument('headless')
# options.add_argument('window-size=1200x600')
# browser = webdriver.Firefox(options=options)

In [ ]:
## SAME CODE

# navigate to page
browser.get('https://mimo-international.com/MIMO/instrument-families.aspx')
browser.implicitly_wait(5)  # wait 5 seconds

# select all text parts from document
elements = browser.find_element_by_id("InstrumentTypeLevel2_exact")
elements = elements.find_elements_by_tag_name("li")# note that if number differs from launch to launch this means better extend wait time
print("Elements found:", len(elements))


In [ ]:
browser.quit()

### NB 
Note, that browser behavior differs for the same code!

## 2.Task 
Our lectures usually have lot's of links. Here are the links to oviginal versions of the documents.

[4. Vector space](https://docs.google.com/presentation/d/1UxjGZPPrPTM_3lCa_gWTk8yZI_qNmTKwtMxr8JZQCIc/edit?usp=sharing)

[6. search trees](https://docs.google.com/presentation/d/1LuZvz3axBD8UuHLagdv0EbhsGEWJmpd7gN5KjwYCp9Y/edit?usp=sharing)

[7-8. Web basics](https://docs.google.com/presentation/d/1bgsCgpjMcQmrFpblRI6oH9SnG4bjyo5SzSSdKxxHNlg/edit?usp=sharing)

Please complete the following tasks:

### 2.1 Inverted index for slides with numbers
I want to type a word, and it should say which slides of which lecture has this word.

In [ ]:
options = webdriver.FirefoxOptions()
options.add_argument('headless')
options.add_argument('window-size=1200x600')

Loading and parsing text and images from google slides

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

def getTextAndImgsFromSlides(url):    
    slides_text = dict() # dictionary slide_num : slide_text
    img_list = [] # list of image urls 
    #TODO: parse google slides and save all text and image urls in slides_text and img_list
    # you should get the contents from ALL slides - however, you will see that at one moment 
    # of time only single slide + few slide previews on the left are visible. To be able to    
    # reach all slides you will need to scroll to and click these previews. While slide contents 
    # can be obtained from previews themselves, speaker notes (which you also have to extract)
    # can be viewed only if a particular slide is open.
    # to scroll the element of interest into view, use can this: 
    # browser.execute_script("arguments[0].scrollIntoView();", el)
    # to click the element, use can use ActionChains library   
    
    # start browser
    browser = webdriver.Firefox(options=options)
    browser.get(url)
    browser.implicitly_wait(5)  # wait 5 seconds
    # get slide thumbnails list
    elements = browser.find_elements_by_css_selector("g.punch-filmstrip-thumbnail")
    # visit every thumbnail, ckick on it to render the slide, and parse required parts 
    counter = 0
    while counter < len(elements):

        for i in range(counter, len(elements)):
            print(i)
            el = elements[i]
            actions = ActionChains(browser)
            # click on a thumbnail to open the slide
            actions.move_to_element(el).click().perform();
            # scroll to element, such that subsequent slides' thumbnails are visible
            browser.execute_script("arguments[0].scrollIntoView();", el)
            # div with speaker notes, let's extract them too
            notes = browser.find_element_by_id("speakernotes")
            # find all text blocks
            txt_subnodes = el.find_elements_by_css_selector("text") + notes.find_elements_by_css_selector("text")
            # first subnode contains slidenumber, so excliding it
            text = " ".join(n.text for n in txt_subnodes[1:])
            slides_text[i+1] = text
            print(text)
            # find all images
            img_subnodes = el.find_elements_by_css_selector("image")
            img_list.extend([n.get_attribute('href') for n in img_subnodes])
        
        counter = i + 1 
        # retrieve elements once again, with new elements made visible due to scrolling
        print("retrieving once more")
        elements = browser.find_elements_by_css_selector("g.punch-filmstrip-thumbnail")
        print(len(elements))
  
    browser.quit()
    return slides_text, img_list

Parsing three presentations

In [ ]:
links = ["https://docs.google.com/presentation/d/1UxjGZPPrPTM_3lCa_gWTk8yZI_qNmTKwtMxr8JZQCIc/edit?usp=sharing", 
         "https://docs.google.com/presentation/d/1LuZvz3axBD8UuHLagdv0EbhsGEWJmpd7gN5KjwYCp9Y/edit?usp=sharing",
         "https://docs.google.com/presentation/d/1bgsCgpjMcQmrFpblRI6oH9SnG4bjyo5SzSSdKxxHNlg/edit?usp=sharing"]

all_imgs = []
all_texts = dict()

for i, link in enumerate(links):
    texts, imgs = getTextAndImgsFromSlides(link)
    all_imgs.extend(imgs)
    all_texts.update([(str(i + 1) + '_' + str(k), v) for (k,v) in texts.items()])

### 2.2 Tests

In [ ]:
texts, imgs = getTextAndImgsFromSlides('http://tiny.cc/00dhkz')

assert len(texts) == 35 # equal to the total number of slides in the presentation 
print(len(texts))

assert len(imgs) > 26 # can be more than that due to visitor icons
print(len(imgs))

assert any("Navigable" in value for value in texts.values()) # word on a slide
assert any("MINUS" in value for value in texts.values()) # word in speaker notes

Building an inverted index and search using it (boolean retrieval is just ok)

In [ ]:
import nltk
from collections import Counter

def build_inverted_index(files_data):
    inverted_index = {}
    for name, strings in files_data.items():
        tokens = nltk.word_tokenize(strings.lower())
        file_index = Counter(tokens)
        # update global index
        for term in file_index.keys():
            file_freq = file_index[term]
            if term not in inverted_index:                
                inverted_index[term] = [file_freq, (name, file_freq)]
            else:
                inverted_index[term][0] += file_freq
                inverted_index[term].append((name, file_freq))
    
    return inverted_index

In [ ]:
inverted_index = build_inverted_index(all_texts)

In [ ]:
def find(query, index):
    # boolean retrieval
    postings = []
    query_tokens = nltk.word_tokenize(query.lower())
    for token in query_tokens:
        if token not in index:  # ignoring absent terms
            continue
        posting = index[token][1:]
        # extract document info only
        posting = [i[0] for i in posting]
        postings.append(posting)
    docs = set.intersection(*map(set,postings))
    return docs

### 2.3 Tests

In [ ]:
queries = ["architecture", "algorithm", "function", "dataset", 
           "Protasov", "cosine", "модель", "например"]

for query in queries:
    r = find(query, inverted_index)
    print("Results for: ", query)
    print("\t", r)
    assert len(r) > 0, "Query should return at least 1 document"
    assert len(r) > 1, "Query should return at least 2 documents"

### 2.4 Images saved
Save all images used in a presentation as separate files.

In [ ]:
#TODO: load and save all images from slides on disk (from one | all presentations, both are fine)
import urllib.request
import os

img_dir = "images"
if not os.path.exists(img_dir):
    os.makedirs(img_dir)

for img_url in all_imgs:
    if not img_url.startswith('//'): # ignoring visitor labels
        urllib.request.urlretrieve(img_url, os.path.join(img_dir, img_url.split('/')[-1] + '.png'))

## 3. Bonus task.  Links exploration

Find all external links in these presentations and index them for search too. I.e., parse them and extend inverted index you built with external links content: when searching by word we should get not only slides that contain it, but also any links that were mentioned in slides and contain this query word. 

In [ ]:
#TODO: bonus task